In [1]:
# -*- coding: utf8 -*-
import numpy as np
import collections
import tensorflow as tf

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
poem_only = []
with open('data/poem.txt', 'r') as f:
    for l in f.readlines():
        p = l.split()[2:]
        for i in range(len(p)):
            p[i] = p[i] + '，' if i % 2 == 0 else p[i] + '。'
        p = ''.join(p)
        poem_only.append(p)
print(poem_only[:2])

['秦川雄帝宅，函谷壮皇居。绮殿千寻起，离宫百雉馀。连薨遥接汉，飞观迥凌虚。云日隐层阙，风烟出绮疏。岩廊罢机务，崇文聊驻辇。玉匣启龙图，金绳披凤篆。韦编断仍续，缥帙舒还卷。对此乃淹留，欹案观坟典。移步出词林，停舆欣武宴。雕弓写明月，骏马疑流电。惊雁落虚弦，啼猿悲急箭。阅赏诚多美，于兹乃忘倦。鸣笳临乐馆，眺听欢芳节。急管韵朱弦，清歌凝白雪。彩凤肃来仪，玄鹤纷成列。去兹郑卫声，雅音方可悦。芳辰追逸趣，禁苑信多奇。桥形通汉上，峰势接云危。烟霞交隐映，花鸟自参差。何如肆辙迹，万里赏瑶池。飞盖去芳园，兰桡游翠渚。萍间日彩乱，荷处香风举。桂楫满中川，弦歌振长屿。岂必汾河曲，方为欢宴所。落日双阙昏，回舆九重暮。长烟散初碧，皎月澄轻素。搴幌玩琴书，开轩引云雾。斜汉耿层阁，清风摇玉树。欢乐难再逢，芳辰良可惜。玉酒泛云罍，兰殽陈绮席。千钟合尧禹，百兽谐金石。得志重寸阴，忘怀轻尺璧。建章欢赏夕，二八尽妖妍。罗绮昭阳殿，芬芳玳瑁筵。佩移星正动，扇掩月初圆。无劳上悬圃，即此对神仙。以兹游观极，悠然独长想。披卷览前踪，抚躬寻既往。望古茅茨约，瞻今兰殿广。人道恶高危，虚心戒盈荡。奉天竭诚敬，临民思惠养。纳善察忠谏，明科慎刑赏。六五诚难继，四三非易仰。广待淳化敷，方嗣云亭响。', '塞外悲风切，交河冰已结。瀚海百重波，阴山千里雪。迥戍危烽火，层峦引高节。悠悠卷旆旌，饮马出长城。寒沙连骑迹，朔吹断边声。胡尘清玉塞，羌笛韵金钲。绝漠干戈戢，车徒振原隰。都尉反龙堆，将军旋马邑。扬麾氛雾静，纪石功名立。荒裔一戎衣，灵台凯歌入。']


In [3]:
print('诗总数：', len(poem_only), '\n' 
      '诗平均长度：', np.average([len(p) for p in poem_only]), '\n' 
      '诗最大长度：', np.max([len(p) for p in poem_only]), '\n')

诗总数： 42974 
诗平均长度： 71.38856052496858 
诗最大长度： 15826 



In [4]:
max_len = 120
poem = [] # 最终用于训练的poem
for p in poem_only:
    if len(p) > max_len:
        s = len(p.split('。')[0]) + 1 # 一句诗单位长度
        c = s * (max_len // s) # 截取的诗长度
        p = p[:c]
        poem.append(p)
    else:
        poem.append(p)
print(poem[:2])

['秦川雄帝宅，函谷壮皇居。绮殿千寻起，离宫百雉馀。连薨遥接汉，飞观迥凌虚。云日隐层阙，风烟出绮疏。岩廊罢机务，崇文聊驻辇。玉匣启龙图，金绳披凤篆。韦编断仍续，缥帙舒还卷。对此乃淹留，欹案观坟典。移步出词林，停舆欣武宴。雕弓写明月，骏马疑流电。', '塞外悲风切，交河冰已结。瀚海百重波，阴山千里雪。迥戍危烽火，层峦引高节。悠悠卷旆旌，饮马出长城。寒沙连骑迹，朔吹断边声。胡尘清玉塞，羌笛韵金钲。绝漠干戈戢，车徒振原隰。都尉反龙堆，将军旋马邑。扬麾氛雾静，纪石功名立。荒裔一戎衣，灵台凯歌入。']


In [5]:
count = [['[', 1000], [']', 1000], ['@', 1000], ['_', 1000]] # [: <GO> ]: <EOS> @: <UNK> _: <PAD>
counter = collections.Counter(''.join(poem)).items()
count.extend(counter)
vocab = [w for w, i in count if i > 2]
int_word = {i: w for i, w in enumerate(vocab)}
word_int = {w: i for i, w in enumerate(vocab)}
vocab_size = len(int_word)
print('字典长度：', vocab_size)
# print(word_int)

字典长度： 5649


In [6]:
trans_poems =[]
for p in poem:
    trans_poem =[]  # translated poem to int
    l = len(p)
    p = '[' + p + ']'
    if l < max_len:
         p = p + '_' * (max_len - l)
    for w in p:
        trans_w = word_int.get(w, word_int['@'])
        trans_poem.append(trans_w)
    trans_poems.append(trans_poem)
sequence_len = np.average([len(p) for p in trans_poems]) - 1
print(trans_poems[:1][:], '\n',
      'input_sequence：', sequence_len)

[[0, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 9, 21, 22, 23, 24, 25, 15, 26, 27, 28, 29, 30, 9, 31, 32, 33, 34, 35, 15, 36, 37, 38, 39, 40, 9, 41, 42, 43, 16, 44, 15, 45, 46, 47, 48, 49, 9, 50, 51, 52, 53, 54, 15, 55, 56, 57, 58, 59, 9, 60, 61, 62, 63, 64, 15, 65, 66, 67, 68, 69, 9, 70, 71, 72, 73, 74, 15, 75, 76, 77, 78, 79, 9, 80, 81, 32, 82, 83, 15, 84, 85, 43, 86, 87, 9, 88, 89, 90, 91, 92, 15, 93, 94, 95, 96, 97, 9, 98, 99, 100, 101, 102, 15, 1]] 
 input_sequence： 121.0


In [21]:
def get_batch(poems, batch_size, batch_num, epochs):
    for i in range(batch_num * epochs):
        n = i % batch_num
        poems = np.asarray(poems).astype(np.int32)
        batch = poems[n * batch_size: (n + 1) * batch_size][:]
        yield batch

In [34]:
batch_size = 64
epochs = 20
batch_num = len(trans_poems) // batch_size
batch = get_batch(trans_poems, batch_size, batch_num, epochs)
print(batch_num)

671


In [9]:
b = batch.__next__()
X, Y = [], []
for i in b:
    X.append(i[:-1])
    Y.append(i[1:])
print(len(b[1][:]), '\n', b[1][:], '\n',
      len(X[1][:]), '\n', X[1][:], '\n',
      len(Y[1][:]), '\n', Y[1][:])

122 
 [  0 103 104 105  41 106   9 107 108 109 110 111  15 112 113  23 114 115
   9 116 117  18 118 119  15  33 120 121 122 123   9  39 124 125 126 127
  15 128 128  74 129 130   9 131  99  43 132 133  15 134 135  26 136 137
   9 138 139  67 140 141  15 142 143 144  55 103   9 145 146 147  60 148
  15 149 150 151 152 153   9 154 155 156 157 158  15 159 160 161  58 162
   9 163 164 165  99 166  15 167 168 169 170 171   9 172 173 174 175 176
  15 177 178 179 180 181   9 182 183 184 185 186  15   1] 
 121 
 [  0 103 104 105  41 106   9 107 108 109 110 111  15 112 113  23 114 115
   9 116 117  18 118 119  15  33 120 121 122 123   9  39 124 125 126 127
  15 128 128  74 129 130   9 131  99  43 132 133  15 134 135  26 136 137
   9 138 139  67 140 141  15 142 143 144  55 103   9 145 146 147  60 148
  15 149 150 151 152 153   9 154 155 156 157 158  15 159 160 161  58 162
   9 163 164 165  99 166  15 167 168 169 170 171   9 172 173 174 175 176
  15 177 178 179 180 181   9 182 183 184 185 186  15

In [10]:
getX = tf.placeholder(tf.int32, shape=[None, 121])
getY = tf.placeholder(tf.int32, shape=[None, 121])

In [28]:
embed_size = 100
rnn_cell_num = 3
rnn_cell_size = 128
num_noise = 10
learning_rate = 0.000001

In [12]:
with tf.variable_scope('embedding', reuse=tf.AUTO_REUSE):
    embedding_matrix = tf.get_variable('embedding_matrix', shape=[vocab_size, embed_size],
                                       initializer=tf.random_uniform_initializer(-1, 1, seed=123))
    embedding = tf.nn.embedding_lookup(embedding_matrix, getX)
with tf.variable_scope('rnn', reuse=tf.AUTO_REUSE):
    lstms = tf.contrib.rnn.MultiRNNCell(
        [tf.contrib.rnn.LSTMCell(rnn_cell_size, tf.random_uniform_initializer(-1, 1, seed=456)) 
         for _ in range(rnn_cell_num)])
    outputs, states = tf.nn.dynamic_rnn(lstms, embedding, dtype=tf.float32)
with tf.variable_scope('NCE', reuse=tf.AUTO_REUSE):
    nce_w = tf.get_variable('nce_w', shape=[vocab_size, rnn_cell_size], 
                       initializer=tf.random_uniform_initializer(-1, 1, seed=234))
    nce_b = tf.get_variable('nce_b', shape=[vocab_size], 
                       initializer=tf.zeros_initializer())
    _getY = tf.reshape(getY, [-1, 1])
    _outputs = tf.reshape(outputs, [-1, rnn_cell_size])
    loss = tf.reduce_mean(
        tf.nn.nce_loss(nce_w, nce_b, inputs=_outputs, labels=_getY, num_sampled=num_noise, num_classes=vocab_size))
with tf.variable_scope('train_op', reuse=tf.AUTO_REUSE):
    optimizer = tf.train.AdamOptimizer(learning_rate)
    tvars = tf.trainable_variables()
    grads, _ = tf.clip_by_global_norm(tf.gradients(loss, tvars), 5)
    global_step = tf.Variable(0, trainable=False, name='global_step', dtype=tf.int32)
    train_op = optimizer.apply_gradients(zip(grads, tvars), global_step=global_step)

In [13]:
tf.summary.histogram('embed', embedding_matrix)
tf.summary.histogram('rnn_cell_state', states)
tf.summary.histogram('nce_w', nce_w)
tf.summary.histogram('nce_b', nce_b)
tf.summary.scalar('loss', loss)
merged_summary_op = tf.summary.merge_all()

In [35]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    summary_writer = tf.summary.FileWriter('./logs/', sess.graph)
    saver = tf.train.Saver()
    checkpoint = tf.train.get_checkpoint_state('./checkpoints/')
    if checkpoint and checkpoint.model_checkpoint_path:
        saver.restore(sess, checkpoint.model_checkpoint_path)
        print('Restored: ', checkpoint.model_checkpoint_path)
    for e in range(epochs):
        for i in range(batch_num):
            b = batch.__next__()
            X, Y = [], []
            for j in b:
                X.append(j[:-1])
                Y.append(j[1:])
            _, cost, summary, g_step = sess.run([train_op, loss, merged_summary_op, global_step],
                                                 feed_dict={getX: X, getY: Y})
            print('Epoch {} Batch {} Iter {} Loss {}'.format(e, i, e*batch_num + i, cost))
            summary_writer.add_summary(summary, g_step)
            if g_step % 100 == 0:
                saver.save(sess, './checkpoints/poem', global_step=g_step)
    summary_writer.close()

INFO:tensorflow:Restoring parameters from ./checkpoints/poem-1700
Restored:  ./checkpoints/poem-1700
Epoch 0 Batch 0 Iter 0 Loss 25.001008987426758
Epoch 0 Batch 1 Iter 1 Loss 20.115055084228516
Epoch 0 Batch 2 Iter 2 Loss 8.75159740447998
Epoch 0 Batch 3 Iter 3 Loss 14.67435073852539
Epoch 0 Batch 4 Iter 4 Loss 21.16809844970703
Epoch 0 Batch 5 Iter 5 Loss 31.040552139282227
Epoch 0 Batch 6 Iter 6 Loss 27.091073989868164
Epoch 0 Batch 7 Iter 7 Loss 14.065450668334961
Epoch 0 Batch 8 Iter 8 Loss 2.7014739513397217
Epoch 0 Batch 9 Iter 9 Loss 33.217594146728516
Epoch 0 Batch 10 Iter 10 Loss 12.829815864562988
Epoch 0 Batch 11 Iter 11 Loss 36.410789489746094
Epoch 0 Batch 12 Iter 12 Loss 15.65182113647461
Epoch 0 Batch 13 Iter 13 Loss 35.03383255004883
Epoch 0 Batch 14 Iter 14 Loss 9.131487846374512
Epoch 0 Batch 15 Iter 15 Loss 14.055779457092285
Epoch 0 Batch 16 Iter 16 Loss 11.477347373962402
Epoch 0 Batch 17 Iter 17 Loss 29.25791358947754
Epoch 0 Batch 18 Iter 18 Loss 25.926746368408

Epoch 0 Batch 165 Iter 165 Loss 31.906679153442383
Epoch 0 Batch 166 Iter 166 Loss 20.083742141723633
Epoch 0 Batch 167 Iter 167 Loss 27.2928409576416
Epoch 0 Batch 168 Iter 168 Loss 31.969839096069336
Epoch 0 Batch 169 Iter 169 Loss 26.21174430847168
Epoch 0 Batch 170 Iter 170 Loss 21.239459991455078
Epoch 0 Batch 171 Iter 171 Loss 14.663291931152344
Epoch 0 Batch 172 Iter 172 Loss 35.607696533203125
Epoch 0 Batch 173 Iter 173 Loss 42.56938171386719
Epoch 0 Batch 174 Iter 174 Loss 27.48258399963379
Epoch 0 Batch 175 Iter 175 Loss 14.284621238708496
Epoch 0 Batch 176 Iter 176 Loss 13.989116668701172
Epoch 0 Batch 177 Iter 177 Loss 24.341188430786133
Epoch 0 Batch 178 Iter 178 Loss 11.03585147857666
Epoch 0 Batch 179 Iter 179 Loss 20.52516746520996
Epoch 0 Batch 180 Iter 180 Loss 31.286291122436523
Epoch 0 Batch 181 Iter 181 Loss 15.603459358215332
Epoch 0 Batch 182 Iter 182 Loss 26.197765350341797
Epoch 0 Batch 183 Iter 183 Loss 19.722183227539062
Epoch 0 Batch 184 Iter 184 Loss 7.8521

KeyboardInterrupt: 

In [37]:
help(tf.train.shuffle_batch)

Help on function shuffle_batch in module tensorflow.python.training.input:

shuffle_batch(tensors, batch_size, capacity, min_after_dequeue, num_threads=1, seed=None, enqueue_many=False, shapes=None, allow_smaller_final_batch=False, shared_name=None, name=None)
    Creates batches by randomly shuffling tensors.
    
    This function adds the following to the current `Graph`:
    
    * A shuffling queue into which tensors from `tensors` are enqueued.
    * A `dequeue_many` operation to create batches from the queue.
    * A `QueueRunner` to `QUEUE_RUNNER` collection, to enqueue the tensors
      from `tensors`.
    
    If `enqueue_many` is `False`, `tensors` is assumed to represent a
    single example.  An input tensor with shape `[x, y, z]` will be output
    as a tensor with shape `[batch_size, x, y, z]`.
    
    If `enqueue_many` is `True`, `tensors` is assumed to represent a
    batch of examples, where the first dimension is indexed by example,
    and all members of `tensors` 